In [1]:
import os, sys
os.environ['ON_PY']="1"
from params import parser
from common_utils import setup_seed
from data_utils import pack_data_from_config
import numpy as np
from test_script.base import Test
import matplotlib.pyplot as plt
import numpy as np
import re
from data_utils import CaseGenerator, SD2_instance_generator, text_to_matrix, matrix_to_text

notebook_dir = os.getcwd()
print(notebook_dir)
# 将工作目录更改为上一级目录
os.chdir("/home/wlc/FJSP-DRL-main")


pygame 2.6.0 (SDL 2.28.4, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html
/home/wlc/FJSP-DRL-main/plot_imgs


In [2]:
def VAE_finetuning_benchmark(args):
    configs = parser.parse_args(args=args)
    # print(configs.hidden_dim_actor, configs.hidden_dim_critic)
    test_model = []
    for model_name in configs.test_model:
        test_model.append((f'./trained_network/{configs.model_source}/{model_name}.pth', model_name))
    print(test_model)

    model = test_model[0]

    data_path = f'./data/{configs.data_source}/{configs.test_data[0]}'
    print(data_path)
    instances_dict = {}
    file_name_list = []
    files = []
    for root, dirs, files in os.walk(data_path):
        # sort files by index
        files.sort(key=lambda s: int(re.findall("\d+", s)[0]))
        files.sort(key=lambda s: int(re.findall("\d+", s)[-1]))
        for f in files:
            file_name_list.append(f)
            g = open(os.path.join(root, f), 'r').readlines()
            
            job_length, op_pt, *args = text_to_matrix(g)
            instances_dict[f] = (job_length, op_pt)

    finetuning_results = []
    for instance in file_name_list:
        job_length, OpPt  = instances_dict[instance]
        ### 对每个实例
        print("\ndata name",instance)
        test = Test(configs, [[job_length], [OpPt]], model[0])
        finetuning_return = test.finetuning(times=10)
        greedy_return = test.greedy_strategy()
        finetuning_min = min(finetuning_return['makespan'])
        greedy = greedy_return[0][0]
        print("min makespan", min(finetuning_min, greedy))
        print(greedy_return[0][0], finetuning_return["makespan"])
        finetuning_results.append(finetuning_return)

    return finetuning_results


In [4]:
model_args = ["--test_model", "10x5",  "--model_source", "SD1"]  #10x5的模型
model_args = ["--test_model", "vari_vae_2025-03-04T17-21-29", "--model_source", "SD2"]
model_args = ["--test_model", "10x5x0+mix", "--model_source", "SD2"]
model_args = ["--test_model", "vari_vae_2025-03-05T09-52-07", "--model_source", "SD1"]
model_args = ["--test_model", "vari_vae_2025-03-05T17-15-39", "--model_source", "SD2"]

for testdata in [
    # "barnes", "behnke", "brandimarte", "dauzere", "fattahi", 
    "Hurink_edata", 
    # "Hurink_rdata", "Hurink_vdata", "kacem"
    ]:
    args = ["--data_source", "BenchData", "--test_data", testdata,  "--network_type", "VAE", *model_args]
    vae_finetuing_result = VAE_finetuning_benchmark(args)

# vae_finetuning_makespans = [_["makespan"] for _ in vae_finetuing_result]
# print(vae_finetuning_makespans)

[('./trained_network/SD2/vari_vae_2025-03-05T17-15-39.pth', 'vari_vae_2025-03-05T17-15-39')]
./data/BenchData/Hurink_edata

data name la01.txt
min makespan 654.0
712.0 [717.0, 674.0, 749.0, 703.0, 717.0, 703.0, 654.0, 686.0, 726.0, 746.0]

data name orb1.txt
min makespan 1099.0
1159.0 [1202.0, 1182.0, 1230.0, 1204.0, 1230.0, 1168.0, 1099.0, 1203.0, 1127.0, 1153.0]

data name car1.txt
min makespan 6986.0
7407.0 [8848.0, 7402.0, 7954.0, 7346.0, 7766.0, 7382.0, 6986.0, 7283.0, 7776.0, 7370.0]

data name orb2.txt
min makespan 946.0
946.0 [1193.0, 956.0, 1013.0, 974.0, 996.0, 1013.0, 970.0, 1042.0, 946.0, 988.0]

data name la02.txt
min makespan 761.0
837.0 [805.0, 852.0, 772.0, 761.0, 809.0, 871.0, 795.0, 837.0, 802.0, 761.0]

data name car2.txt
min makespan 7674.0
7674.0 [7988.0, 7819.0, 7819.0, 7819.0, 7674.0, 7867.0, 8189.0, 7674.0, 7819.0, 7819.0]

data name orb3.txt
min makespan 1061.0
1155.0 [1315.0, 1246.0, 1207.0, 1166.0, 1079.0, 1166.0, 1128.0, 1144.0, 1061.0, 1193.0]

data name la

In [ ]:
model_args = ["--test_model", "vari_vae_SD1_2025-03-08T19-44-24", "--model_source", "SD1"]

for testdata in ["barnes", "behnke", "brandimarte", "dauzere", "fattahi", "Hurink_edata", "Hurink_rdata", "Hurink_vdata", "kacem"]:
    args = ["--data_source", "BenchData", "--test_data", testdata,  "--network_type", "VAE", *model_args]
    vae_finetuing_result = VAE_finetuning_benchmark(args)


[('./trained_network/SD1/vari_vae_SD1_2025-03-08T19-44-24.pth', 'vari_vae_SD1_2025-03-08T19-44-24')]
./data/BenchData/barnes

data name mt10c1.txt
min makespan 1063.0
1113.0 [1170.0, 1117.0, 1126.0, 1063.0, 1091.0, 1113.0, 1117.0, 1113.0, 1113.0, 1113.0]

data name setb4xy.txt
min makespan 1053.0
1074.0 [1141.0, 1075.0, 1053.0, 1068.0, 1068.0, 1066.0, 1053.0, 1068.0, 1069.0, 1079.0]

data name setb4xxx.txt
min makespan 1053.0
1068.0 [1297.0, 1087.0, 1060.0, 1064.0, 1056.0, 1074.0, 1068.0, 1053.0, 1068.0, 1053.0]

data name setb4cc.txt
min makespan 1029.0
1044.0 [1233.0, 1083.0, 1084.0, 1050.0, 1084.0, 1044.0, 1055.0, 1029.0, 1054.0, 1076.0]

data name setb4x.txt
min makespan 1038.0
1038.0 [1242.0, 1101.0, 1073.0, 1054.0, 1050.0, 1079.0, 1208.0, 1107.0, 1038.0, 1065.0]

data name setb4xx.txt
min makespan 1060.0
1075.0 [1307.0, 1079.0, 1090.0, 1068.0, 1068.0, 1060.0, 1076.0, 1065.0, 1079.0, 1075.0]

data name setb4xyz.txt
min makespan 1022.0
1044.0 [1220.0, 1022.0, 1026.0, 1038.0, 1075.0